In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [2]:
from keras.datasets import cifar10
# let's load data 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
#normalizing inputs from 0-255 to 0.0-1.0 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train = X_train / 255.0 
X_test = X_test / 255.0

In [4]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [5]:
# one hot encode outputs 
from keras.utils import np_utils 

y_train = np_utils.to_categorical(y_train) 
y_test = np_utils.to_categorical(y_test) 
num_classes = y_test.shape[1]

In [6]:
import keras
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import backend as K 

In [7]:
model = Sequential()

# 2D concolutional layer.32 == filters needed.(an array of numeric values). (3,3)==size of the filter(3 r, 3 c)
# 32,32,3 - shape of data-h,w,rgb values(depth/intensity)
# relu activation replaces -ve pixel values with 0s.
model.add(Conv2D(32, (3, 3), input_shape=(32,32,3), activation='relu', padding='same'))  #output=feature maps
# dropout used to prevent overfitting
model.add(Dropout(0.2)) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
# maxpooling==dim reduction - while retaining most important features
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
# flatten used to convert feature map to 1D.
model.add(Flatten())
model.add(Dropout(0.2)) 
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3))) 
model.add(Dropout(0.2)) 
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3))) 
model.add(Dropout(0.2)) 
# softmax used - multiclass classification
model.add(Dense(num_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

In [9]:
# model compilation
learning_rate = 0.01
epochs = 50 # we want a high quality model 
decay = learning_rate/epochs 
sgd = SGD(lr=learning_rate, momentum=0.9, decay=decay, nesterov=False) #optimisation function - stochastic gradient descent
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# model training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32) 

Epoch 1/50
1563/1563 [==============================] - 445s 285ms/step - loss: 1.8494 - accuracy: 0.3181 - val_loss: 1.5454 - val_accuracy: 0.4361
Epoch 2/50
1563/1563 [==============================] - 449s 287ms/step - loss: 1.4230 - accuracy: 0.4830 - val_loss: 1.2618 - val_accuracy: 0.5440
Epoch 3/50
1563/1563 [==============================] - 448s 287ms/step - loss: 1.2208 - accuracy: 0.5586 - val_loss: 1.1460 - val_accuracy: 0.5877
Epoch 4/50
1563/1563 [==============================] - 444s 284ms/step - loss: 1.0606 - accuracy: 0.6231 - val_loss: 1.0308 - val_accuracy: 0.6394
Epoch 5/50
1563/1563 [==============================] - 447s 286ms/step - loss: 0.9396 - accuracy: 0.6675 - val_loss: 0.9365 - val_accuracy: 0.6679
Epoch 6/50
1563/1563 [==============================] - 444s 284ms/step - loss: 0.8435 - accuracy: 0.7010 - val_loss: 0.8294 - val_accuracy: 0.7123
Epoch 7/50
1563/1563 [==============================] - 444s 284ms/step - loss: 0.7622 - accuracy: 0.7321 - val_

In [ ]:
# Final evaluation of the model 
scores = model.evaluate(X_test, y_test, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))